In [ ]:
# importing all relevant libraries to process our text
import spacy
import re
import string
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.model_selection import train_test_split
import fasttext

In [3]:
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm') # use the desired spacy model. 'sm' small, 'md' medium, 'lg' large

In [20]:
# preprocessing function to clean our text data and get it in a specific format
def clean_text(text):
    if not isinstance(text,str):
        return ""
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.lower()  # Lowercase
    text = text.translate(str.maketrans('','', string.punctuation))  # Remove punctuation
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s{2,}',' ',text) # Remove extra spaces
    text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags
    #doc = nlp(text) # To tokenize the text
    #doc1 = [token for token in doc if not token.is_stop] # To remove stop words
    #doc2 = [token.lemma_ for token in doc1] # To get the lemmatized text
    return text

In [38]:
df = pd.read_csv("twitter_training.csv", names=["index","game","sentiment","description"],header = None) # Load our dataset
#df = df.apply(clean_text) # Preprocess the text in the whole dataframe
#df['game'] = df['game'].apply(lambda game: clean_text(game)) # Preprocess the text in each column
df['sentiment'] = df["sentiment"].apply(lambda sentiment: clean_text(sentiment)) # Preprocess the text in each column
df['description'] = df["description"].apply(lambda sentiment: clean_text(sentiment) )# Preprocess the text in each column

In [40]:
#df["game"] = "__label__" + df["game"] # creating the labels for the fasttext format file
df["sentiment"] = "__label__" + df["sentiment"] # creating the labels for the fasttext format file
df["label_desc"] =  df["sentiment"] + " " + df["description"] # Creating the column with the whole data for the fasttext file

In [45]:
train , test = train_test_split(df.label_desc, test_size=0.2)

In [48]:
train.to_csv("sentiment.txt",header=False,index=False)
test.to_csv("sentiment2.txt",header=False,index=False)

In [ ]:
model = fasttext.train_supervised(input="/content/sentiment.txt",epoch=35, lr=1.0, wordNgrams=2, verbose=2) # Training the model
model.test("/content/sentiment2.txt") # Testing the model

In [ ]:
model.save_model("sentiment_analysis_model.bin") # For saving the model